In [1]:
import torch
import itertools
from torch.utils.data import DataLoader

from dataset import FontDataset
from models import Discriminator, ContentEncoder, StyleEncoder, Decoder
from losses import DisLoss
from accelerate import Accelerator

In [2]:
lr = 0.0002
beta1 = 0.5

In [3]:
accelerator = Accelerator()
dataset = FontDataset(path="../data/raw/")
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
discriminator = Discriminator()
content_encoder = ContentEncoder(G_ch=64)
style_encoder = StyleEncoder(G_ch=64)
decoder = Decoder(G_ch=64, nEmbedding=1024)

Param count for Ds initialized parameters: 19541696
Param count for Ds initialized parameters: 20591296
Param count for Ds initialized parameters: 27289027


In [4]:
optimizer_G = torch.optim.Adam(itertools.chain(
    content_encoder.parameters(),
    style_encoder.parameters(),
    decoder.parameters()), lr=lr, betas=(beta1, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))

criterionD = DisLoss()

In [5]:
dataloader, content_encoder, style_encoder, decoder, discriminator, optimizer_D, optimizer_G = accelerator.prepare(
    dataloader, content_encoder, style_encoder, decoder, discriminator, optimizer_D, optimizer_G)

In [6]:
for data in dataloader:
    optimizer_D.zero_grad()
    optimizer_G.zero_grad()

    cont, residulte_features = content_encoder(data['content'])
    style_emb, style_fc, residual_features_style = style_encoder(data['style'])
    img_print2write = decoder(cont, residulte_features, style_emb, style_fc, residual_features_style)

    real_out = discriminator(data['target'])
    fake_out = discriminator(img_print2write)

    loss_D = criterionD(real_out, True) + criterionD(fake_out, False)
    loss_G = criterionD(fake_out, True)

    accelerator.backward(loss_D)
    optimizer_D.step()

    accelerator.backward(loss_G)
    optimizer_G.step()


RuntimeError: MPS backend out of memory (MPS allocated: 5.91 GB, other allocations: 536.91 MB, max allowed: 6.77 GB). Tried to allocate 512.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [10]:
import os
os.getcwd()

'/Users/taeseoklim/PycharmProject/font/ai_font/leo-font'

In [11]:
curdir = os.path.dirname(os.path.abspath(__file__))

NameError: name '__file__' is not defined

In [2]:
from configs import DefaultConfig

In [3]:
config = DefaultConfig()

In [4]:
config.curfile

'configs.py'

In [6]:
import os
os.path.basename(__file__)

NameError: name '__file__' is not defined

In [11]:
import torch
import numpy as np
from datetime import datetime
from PIL import Image


In [6]:
a = torch.ones([2,2])

In [9]:
torch.ones_like(a)

tensor([[1., 1.],
        [1., 1.]])

In [15]:
np.ones(1).astype(bool)

array([ True])

In [12]:
import os
os.chdir()

TypeError: chdir() missing required argument 'path' (pos 1)

In [17]:
import os
import pandas as pd
import random
from PIL import Image

import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms

t = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5])
        ])

In [18]:
a = t(Image.open("../data/raw_png_128/UhBee BEOJJI__6114.png").convert("RGB"))

In [19]:
torch.max(a)

tensor(1.)

In [20]:
torch.min(a)

tensor(0.)